Full pandas dataframe created for all shep race results

todo:
create graph of box number vs place
create lin model of distance box vs time/place?

idea:
clustering analysis of dog to predict dogs that will behave the same  

working:
currently working with all split time correct in r
creating dog index in r to hold previous race results


In [1]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import tqdm
import pickle

In [2]:
#Loading race results database
os.chdir(r'C:\Users\Nick\Documents\GitHub\grvmodel\DATA\GRV_scraper\full race results')
file_list = os.listdir()
path = os.getcwd()


file_list_new = [x for x in file_list if ".xml" in x]
file_list_new

['rr-BAL01Apr17.xml',
 'rr-BAL01Apr19.xml',
 'rr-BAL01Apr22.xml',
 'rr-BAL01Aug16.xml',
 'rr-BAL01Aug18.xml',
 'rr-BAL01Dec18.xml',
 'rr-BAL01Feb16.xml',
 'rr-BAL01Feb17.xml',
 'rr-BAL01Feb21.xml',
 'rr-BAL01Jan18.xml',
 'rr-BAL01Jan20.xml',
 'rr-BAL01Jul19.xml',
 'rr-BAL01Mar17.xml',
 'rr-BAL01Mar21.xml',
 'rr-BAL01May17.xml',
 'rr-BAL01Nov17.xml',
 'rr-BAL01Nov21.xml',
 'rr-BAL01Oct18.xml',
 'rr-BAL01Sep19.xml',
 'rr-BAL01Sep21.xml',
 'rr-BAL02Apr18.xml',
 'rr-BAL02Aug19.xml',
 'rr-BAL02Aug21.xml',
 'rr-BAL02Dec16.xml',
 'rr-BAL02Dec17.xml',
 'rr-BAL02Dec19.xml',
 'rr-BAL02Feb22.xml',
 'rr-BAL02Jan17.xml',
 'rr-BAL02Jan19.xml',
 'rr-BAL02Jul18.xml',
 'rr-BAL02Jun21.xml',
 'rr-BAL02Mar22.xml',
 'rr-BAL02May17.xml',
 'rr-BAL02Nov16.xml',
 'rr-BAL02Oct17.xml',
 'rr-BAL02Oct19.xml',
 'rr-BAL02Oct21.xml',
 'rr-BAL03Apr17.xml',
 'rr-BAL03Apr19.xml',
 'rr-BAL03Aug16.xml',
 'rr-BAL03Aug17.xml',
 'rr-BAL03Dec18.xml',
 'rr-BAL03Feb16.xml',
 'rr-BAL03Feb21.xml',
 'rr-BAL03Jan18.xml',
 'rr-BAL03

In [3]:
def result_labels(file_list, path):
    label_list = []
    for i in file_list:
        root = ET.parse(i).getroot()
        
        for race in root.iter('Race'):
            race_id = race.attrib["id"]
            print("race id = ",race_id)
            
            places = []
                        
            for dog in race.iter('Dog'):
                places.append(dog[0].text)
            print(places)
        break
            

In [4]:
def result_getter(file_list, path):
    master_list = []

    for i in tqdm.tqdm(file_list):
        root = ET.parse(i).getroot()

        track = root[0]
        date = root[1]

        for race in root.iter("Race"):
            places = []
            ids = []
            boxes = []
            splittimes = []
            split_margin = []
            weights = []
            margins = []
            split_position = []
            runtimes = []
            start_price = []
            dog_name = []

            for dog in race.iter("Dog"):
                if not dog.attrib["id"]:
                    continue
                if not dog[0].text:
                    continue
                if (dog[0].text in "RSBFNPTBD") or ("=" in dog[0].text):
                    # print("scratched check working")
                    continue
                else:
                    places.append(dog[0].text)
                    ids.append(dog.attrib["id"])
                    boxes.append(dog[3].text)
                    splittimes.append(dog[13].text)
                    runtimes.append(dog[14].text)
                    weights.append(dog[5].text)
                    dog_name.append(dog[1].text)
                    if dog[6].text:
                        start_price.append(dog[6].text[1:-1])
                    if dog[0].text == "1":
                        margins.append(0)
                    else:
                        margins.append(dog[8].text)

            for i in range(len(splittimes)):
                if splittimes[0] == None:
                    split_margin.append(pd.NaT)
                    continue
                try:
                    split_margin.append(
                        "%.2f" % (float(splittimes[i]) - float(min(splittimes)))
                    )

                    # trying to get split position
                except:
                    split_margin.append(pd.NaT)

            zipall = list(
                zip(
                    places,
                    ids,
                    boxes,
                    splittimes,
                    split_margin,
                    runtimes,
                    weights,
                    margins,
                    start_price,
                    dog_name
                )
            )
            # print(zipall)
            for i in range(len(zipall)):
                zipall[i] = (
                    race.attrib["id"],
                    date.text,
                    track.text,
                    race[3].text,
                    race[4].text,
                ) + zipall[i]
                master_list.append(zipall[i])

    return master_list


In [5]:
master_list = result_getter(file_list_new, path)
#master_list

100%|██████████| 5812/5812 [00:46<00:00, 124.02it/s]


In [6]:
col_names = ["race_id", "date", "track","dist","grade", "place", "dog_id", "box", "split_times", "split_margins", "run_time", "weigth", "margin", "start_price", "dog_name"]
df = pd.DataFrame(master_list, columns = col_names)
df.place = df.place.astype(float)
df.start_price = df.start_price.astype(float)
df.place.unique()

array([1., 2., 3., 4., 5., 6., 7., 8.])

In [7]:
print(df.size)
df = df.dropna()
print(df.size)
df.to_csv("masterfile.csv")

with open("results-df.npy", "wb") as fp:   #Pickling
    
    pickle.dump(df, fp)

7467495
7445475


In [8]:
df.grade.value_counts()

Grade 5                            111414
Mixed 4/5                           45205
Maiden                              39769
Grade 5 T3                          33416
Mixed 6/7                           27616
Restricted Win                      20880
Grade 7                             20614
Grade 5 Heat                        16703
Tier 3 - Maiden                     15605
Maiden Heat                         13994
Restricted Win Heat                 13184
Tier 3 - Grade 7                    12681
Grade 6                             12206
S/E Heat                            11190
Free For All                        10290
Mixed 6/7 Heat                       9770
Tier 3 - Restricted Win              8897
Grade 5 No Penalty                   8320
Tier 3 - Grade 6                     7532
Grade 5 Final                        5426
Aged Prize Money                     5367
Special Event                        5304
Mixed 3/4                            4590
Restricted Win Final              

In [9]:
df.track.value_counts()

Ballarat         48727
Shepparton       48564
Warragul         48466
Bendigo          46188
Healesville      44860
Geelong          43661
Sale             30253
Cranbourne       30206
Warrnambool      28631
Traralgon        24520
The Meadows      22504
Sandown Park     21640
Meadows (MEP)    19477
Horsham          19354
Sandown (SAP)    19314
Name: track, dtype: int64

In [10]:
#generates adjusted place to transfer to form results

dfsorted = df.sort_values(['box'])
dfsorted['place_adj'] = 1/(dfsorted["place"])
grouped = dfsorted.groupby(['race_id'])

for group in grouped:
    
    print(group[1])
    break
 

       race_id       date     track  dist       grade  place     dog_id box  \
175  139100145  01 Aug 16  Ballarat  390m  Grade 5 T3    1.0    -729107   1   
176  139100145  01 Aug 16  Ballarat  390m  Grade 5 T3    2.0    -723904   2   
182  139100145  01 Aug 16  Ballarat  390m  Grade 5 T3    8.0  887800060   3   
177  139100145  01 Aug 16  Ballarat  390m  Grade 5 T3    3.0  275790135   4   
180  139100145  01 Aug 16  Ballarat  390m  Grade 5 T3    6.0  744220010   5   
179  139100145  01 Aug 16  Ballarat  390m  Grade 5 T3    5.0    -751058   6   
178  139100145  01 Aug 16  Ballarat  390m  Grade 5 T3    4.0  219770099   7   
181  139100145  01 Aug 16  Ballarat  390m  Grade 5 T3    7.0  256990051   8   

    split_times split_margins run_time weigth margin  place_adj  
175        8.62          0.11    22.57   34.5   0.90   1.000000  
176        8.51          0.00    22.64   28.6   0.90   0.500000  
182        8.84          0.33    23.01   25.8   6.21   0.125000  
177        8.63         

In [11]:
dfsorted.to_csv("masterfilenew.csv")


In [25]:
winners = df[df["place"]==1.0]